In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from functools import partial

%matplotlib inline

In [ ]:
import logging
logging.basicConfig(level=logging.CRITICAL)

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 14

# ELFI - Engine for Likelihood-Free Inference


European Meeting of Statisticians
July 25th, 2017

Antti Kangasrääsiö
Aalto University, Probabilistic Machine Learning Research Group

Joint work with:
Jarno Lintusaari (Aalto), Henri Vuollekoski (Aalto), Kusti Skytén (Aalto), Marko Järvenpää (Aalto), Michael Gutmann (University of Edinburgh), Aki Vehtari (Aalto), Jukka Corander (University of Oslo), Samuel Kaski (Aalto)

## Example: the 2nd order Moving Average model (MA2)

$$
y_t = w_t + \theta_1 w_{t-1} + \theta_2 w_{t-2}
$$

In [ ]:
def MA2(t1, t2, n_obs=100, batch_size=1, random_state=None):
    t1 = np.atleast_2d(t1).reshape(-1,1)
    t2 = np.atleast_2d(t2).reshape(-1,1)
    w = random_state.randn(batch_size, n_obs+2)
    y = w[:, 2:] + (t1 * w[:, 1:-1]) + (t2 * w[:, :-2])
    return y 

### Generate some "observed" data:

In [ ]:
# true parameters
t1_true = 0.6
t2_true = 0.2

# Set up observed data y with some random seed
rs = np.random.RandomState(2017)
y_obs = MA2(t1_true, t2_true, random_state=rs)

# Plot the observed sequence
plt.figure(figsize=(11, 6));
plt.plot(y_obs.flatten());

### To illustrate the stochasticity, let's plot a couple of more observations with the same true parameters:

In [ ]:
plt.figure(figsize=(11, 6));
plt.plot(y_obs.flatten());
plt.plot(MA2(t1_true, t2_true, random_state=rs).flatten());
plt.plot(MA2(t1_true, t2_true, random_state=rs).flatten());

### How can you compare these?

### Comparing noisy data

Calculate the discrepancy (distance) between some summary statistics, like the first two autocovariances:

In [ ]:
def autocov(x, lag=1):
    mu = np.mean(x, axis=1, keepdims=True)
    C = np.mean(x[:,lag:] * x[:,:-lag], axis=1, keepdims=True) - mu**2.
    return C

def distance(*simulated, observed):
    d = np.linalg.norm(np.array(simulated) - np.array(observed),
                       ord=2, axis=0)
    return d

And then what?

In [ ]:
import elfi

## Using ELFI to infer the parameters

In [ ]:
t1 = elfi.Prior(scipy.stats.uniform, 0, 2)
t2 = elfi.Prior('uniform', 0, 2)

In [ ]:
Y = elfi.Simulator(MA2, t1, t2, observed=y_obs)

In [ ]:
S1 = elfi.Summary(autocov, Y)

autocov2 = partial(autocov, lag=2)
S2 = elfi.Summary(autocov2, Y)

In [ ]:
d = elfi.Discrepancy(distance, S1, S2)

In [ ]:
elfi.draw(d)

### We could proceed... but being equal is not always nice

In [ ]:
class CustomPrior_t1(elfi.Distribution):
    def rvs(b, size=1, random_state=None):
        u = scipy.stats.uniform.rvs(loc=0, scale=1, size=size,
                                    random_state=random_state)
        t1 = np.where(u<0.5, np.sqrt(2.*u)*b-b, -np.sqrt(2.*(1.-u))*b+b)
        return t1
    
    def pdf(x, b):
        p = 1./b - np.abs(x) / (b*b)
        p = np.where(p < 0., 0., p)
        return p

class CustomPrior_t2(elfi.Distribution):
    def rvs(t1, a, size=1, random_state=None):
        locs = np.maximum(-a-t1, t1-a)
        scales = a - locs
        t2 = scipy.stats.uniform.rvs(loc=locs, scale=scales, size=size, 
                                     random_state=random_state)
        return t2
    
    def pdf(x, t1, a):
        locs = np.maximum(-a-t1, t1-a)
        scales = a - locs
        p = scipy.stats.uniform.pdf(x, loc=locs, scale=scales)
        p = np.where(scales>0., p, 0.)
        return p

### These priors sample from a triangle

$-2<\theta_1<2$ with $\theta_1+\theta_2>-1$ and $\theta_1-\theta_2<1$

In [ ]:
t1_1000 = CustomPrior_t1.rvs(2, 1000)
t2_1000 = CustomPrior_t2.rvs(t1_1000, 1, 1000)
plt.scatter(t1_1000, t2_1000, s=8, edgecolor='none');

### Redefine the model with these priors

In [ ]:
elfi.new_model()

t1 = elfi.Prior(CustomPrior_t1, 2)
t2 = elfi.Prior(CustomPrior_t2, t1, 1)
Y = elfi.Simulator(MA2, t1, t2, observed=y_obs)
S1 = elfi.Summary(autocov, Y)
S2 = elfi.Summary(autocov2, Y)
d = elfi.Discrepancy(distance, S1, S2)

elfi.draw(d)

### Now we just sample *a lot* to get an approximate posterior

In [ ]:
# make sure we have the native client
from elfi.clients.native import set_as_default
set_as_default()

In [ ]:
rej = elfi.Rejection(d, [t1, t2], batch_size=10000)

%time result = rej.sample(100000, n_sim=1000000)

In [ ]:
result.summary()

In [ ]:
ax = result.plot_marginals();
ax[0].axvline(t1_true, color='r');
ax[1].axvline(t2_true, color='r');

In [ ]:
result.plot_pairs(s=8);

## Let's do that in parallel!

Currently ELFI supports the powerful *ipyparallel* library for parallel and distributed computing.

In [ ]:
from elfi.clients.multiprocessing import set_as_default
set_as_default()

In [ ]:
rej = elfi.Rejection(d, [t1, t2], batch_size=10000)

%time result_parallel = rej.sample(100000, n_sim=1000000)

In [ ]:
result_parallel.summary()

## Sequential Monte Carlo ABC (Importance sampling)

In [ ]:
smc = elfi.SMC(d, [t1, t2], batch_size=10000)

schedule = [0.7, 0.2, 0.05]

result_smc = smc.sample(1000, schedule)

In [ ]:
result_smc.summary()

In [ ]:
result_smc.plot_marginals(all=True, bins=25, figsize=(6, 1), fontsize=12)

## BOLFI (Bayesian Optimization for Likelihood-Free Inference)

In [ ]:
log_d = elfi.Operation(np.log, d)

In [ ]:
bolfi = elfi.BOLFI(log_d, batch_size=5, initial_evidence=20, update_interval=10, 
                   bounds={'t1':(-2, 2), 't2':(-1, 1)}, acq_noise_var=[0.1, 0.1])
post = bolfi.fit(n_evidence=100)

### ELFI currently uses GPy for Gaussian processes

In [ ]:
bolfi.target_model

In [ ]:
bolfi.plot_state()

In [ ]:
# ignore plotting warning
import warnings
warnings.simplefilter("ignore")

In [ ]:
bolfi.target_model._gp.plot();

In [ ]:
post.plot(logpdf=False)

In [ ]:
%time result_bolfi = bolfi.sample(10)

In [ ]:
result_bolfi.plot_traces();

In [ ]:
result_bolfi.summary()